In [6]:
!python -V

Python 3.10.11


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [ ]:
# import mlflow


# mlflow.set_tracking_uri("sqlite:///mlflow.db")
# mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [17]:
import mlflow
from azureml.core import Workspace, Experiment

# Connect to Azure ML workspace
ws = Workspace.from_config()

# OPTIONAL: You can still log MLflow params and metrics
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# Define the Azure ML Experiment (instead of mlflow.set_experiment)
experiment = Experiment(ws, "nyc-taxi-experiment")

In [19]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [20]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [21]:
len(df_train), len(df_val)

(73908, 61921)

In [22]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [23]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [24]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [27]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715207559742

In [29]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
# with mlflow.start_run():

#     mlflow.set_tag("developer", "cristian")

#     mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#     mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

#     alpha = 0.1
#     mlflow.log_param("alpha", alpha)
#     lr = Lasso(alpha)
#     lr.fit(X_train, y_train)

#     y_pred = lr.predict(X_val)
#     rmse = mean_squared_error(y_val, y_pred, squared=False)
#     mlflow.log_metric("rmse", rmse)

#     mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [34]:
import mlflow
from azureml.core import Workspace, Experiment

# Connect to Azure ML workspace
ws = Workspace.from_config()

# Set MLflow to track to Azure ML backend
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# Define the Azure ML experiment
experiment = Experiment(ws, "nyc-taxi-experiment")

from azureml.core import Workspace, Experiment

# Connect to Azure ML workspace
ws = Workspace.from_config()

# Define the experiment
experiment = Experiment(ws, "nyc-taxi-experiment")

# Start Azure ML run
with experiment.start_logging() as run:

    # Use Azure ML run to set tag (instead of mlflow.set_tag)
    run.tag("developer", "cristian")

    # Log parameters
    # Log parameters (MLflow works here)
    run.log("train-data-path", "'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet'")
    run.log("valid-data-path", "'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet'")
    alpha = 0.1
    run.log("alpha", alpha)

    # Train model
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    run.log("rmse", rmse)

    # Upload artifact (e.g., pickled model)
    run.upload_file(
        name="models_pickle/lin_reg.bin",    # target path in Azure ML run
        path_or_stream="models/lin_reg.bin"  # local path
    )

    print(f"Run ID: {run.id}")

Run ID: 3b0ab851-ac85-42cb-92f5-33c5436b7044


In [35]:
import xgboost as xgb

In [36]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [37]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag("model", "xgboost")
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=1000,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=50
#         )
#         y_pred = booster.predict(valid)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)

#     return {'loss': rmse, 'status': STATUS_OK}

In [42]:
from azureml.core import Workspace, Experiment
from hyperopt import STATUS_OK

# Connect to Azure ML workspace
ws = Workspace.from_config()
experiment = Experiment(ws, "nyc-taxi-xgboost")

def objective(params):
    with experiment.start_logging() as run:
        # Log tags (Azure ML)
        run.tag("model", "xgboost")
        
        # Log parameters (Azure ML)
        for param_name, param_value in params.items():
            run.log(param_name, param_value)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        
        # Predict + calculate RMSE
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        run.log("rmse", rmse)
        
        # OPTIONAL: Save booster model locally and upload as artifact
        booster.save_model("outputs/xgboost_model.json")
        run.upload_file(name="models/xgboost_model.json", path_or_stream="outputs/xgboost_model.json")
        
        print(f"Run ID: {run.id}, RMSE: {rmse}")
    
    return {'loss': rmse, 'status': STATUS_OK}


In [43]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 10, 3)),
    'learning_rate': hp.loguniform('learning_rate', -2, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -2, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -2, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 1),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[0]	validation-rmse:11.06025                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           


/anaconda/envs/mlops_course/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:56:59] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:10.13273                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           


/anaconda/envs/mlops_course/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:57:09] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:7.31645                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


...

[1]	validation-rmse:9.58206                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


/anaconda/envs/mlops_course/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:58:19] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.87520                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


/anaconda/envs/mlops_course/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:58:31] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.85588                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


In [45]:
# Connect to your experiment
experiment = Experiment(workspace=ws, name="nyc-taxi-xgboost")

# Get all runs
runs = list(experiment.get_runs())

# Initialize tracking variables
best_rmse = float('inf')
best_run = None

# Loop through runs to find the one with the lowest RMSE
for run in runs:
    metrics = run.get_metrics()
    if "rmse" in metrics:
        rmse = metrics["rmse"]
        if rmse < best_rmse:
            best_rmse = rmse
            best_run = run

# Print best run details and parameters
if best_run:
    print(f"✅ Best Run ID: {best_run.id}")
    print(f"✅ Best RMSE: {best_rmse}")

    # Get all logged metrics (includes params because we logged them as run.log)
    best_metrics = best_run.get_metrics()
    
    # Filter out non-hyperparameter metrics if needed
    hyperparams = {k: v for k, v in best_metrics.items() if k != "rmse"}
    print("✅ Best Parameters:")
    for param, value in hyperparams.items():
        print(f"  {param}: {value}")
else:
    print("No runs with RMSE found.")


✅ Best Run ID: 035299ab-8c08-437d-ac8f-87c8889db8ed
✅ Best RMSE: 6.50525177877115
✅ Best Parameters:
  learning_rate: 0.6487713208258669
  min_child_weight: 1.2039992026692996
  max_depth: 9
  objective: reg:linear
  reg_alpha: 0.280008415771942
  reg_lambda: 0.19354193303148148
  seed: 42


In [46]:
hyperparams

{'learning_rate': 0.6487713208258669,
 'min_child_weight': 1.2039992026692996,
 'max_depth': 9,
 'objective': 'reg:linear',
 'reg_alpha': 0.280008415771942,
 'reg_lambda': 0.19354193303148148,
 'seed': 42}

In [18]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [ ]:
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# from sklearn.svm import LinearSVR

# mlflow.sklearn.autolog()

# for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

#     with mlflow.start_run():

#         mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
#         mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
#         mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

#         mlmodel = model_class()
#         mlmodel.fit(X_train, y_train)

#         y_pred = mlmodel.predict(X_val)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)
        

In [48]:
import pickle
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from azureml.core import Workspace, Experiment

# Connect to Azure ML
ws = Workspace.from_config()
experiment = Experiment(ws, "nyc-taxi-sklearn")

# Define model classes to loop over
model_classes = [
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    LinearSVR
]

for model_class in model_classes:

    with experiment.start_logging() as run:
        
        model_name = model_class.__name__
        print(f"🔄 Training {model_name}...")

        # Log metadata
        run.log("model_name", model_name)
        run.log("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
        run.log("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

        # Train model
        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        run.log("rmse", rmse)
        print(f"✅ {model_name} RMSE: {rmse}")

        # Save model locally
        model_output_path = f"outputs/{model_name}_model.pkl"
        with open(model_output_path, "wb") as f_out:
            pickle.dump(mlmodel, f_out)

        # Upload model as artifact to Azure ML
        run.upload_file(
            name=f"models/{model_name}_model.pkl",        # Path in Azure ML outputs
            path_or_stream=model_output_path              # Local file path
        )

        print(f"📦 Saved and uploaded {model_name} model to Azure ML.")

print("🎉 All models trained, logged, and uploaded.")


🔄 Training RandomForestRegressor...
✅ RandomForestRegressor RMSE: 6.906262939313015
📦 Saved and uploaded RandomForestRegressor model to Azure ML.


SnapshotException: SnapshotException:
	Message: Failed to take a snapshot of .. Pass snapshot_directory=None to start_logging to skip taking a snapshot of the given directory.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Failed to take a snapshot of .. Pass snapshot_directory=None to start_logging to skip taking a snapshot of the given directory."
    }
}